In [2]:
import sys
sys.path.append("..")
from global_config import ORIGIN_ANNO_DIR_PATH

In [3]:
from PIL import Image
import os

folder = ORIGIN_ANNO_DIR_PATH
files = [f for f in os.listdir(folder) if f.lower().endswith(('.png', '.jpg'))]

p_mode = []
not_p_mode = []

for f in files:
    path = os.path.join(folder, f)
    try:
        img = Image.open(path)
        if img.mode == 'P':
            p_mode.append(f)
        else:
            not_p_mode.append((f, img.mode))
    except Exception as e:
        print(f"Error reading {f}: {e}")

# print("✅ P mode images:")
# for f in p_mode:
#     print("  ", f)

print("\n❌ Not P mode images:")
for f in not_p_mode:
    print("  ", f)



❌ Not P mode images:


In [3]:
import numpy as np
from tqdm import tqdm

palette = [
        [0, 0, 0], [204, 0, 0], [76, 153, 0], [204, 204, 0], [51, 51, 255], [204, 0, 204], 
        [0, 255, 255], [255, 204, 204], [102, 51, 0], [255, 0, 0], 
        [102, 204, 0], [255, 255, 0], [0, 0, 153], [0, 0, 204], [255, 51, 153], 
        [0, 204, 204], [0, 51, 0], [255, 153, 51], [0, 204, 0]
]
palette = np.array(palette)

def rgb2mask(img_array, palette):
    h, w, _ = img_array.shape
    mask = np.zeros((h, w), dtype=np.uint8)
    for idx, color in enumerate(palette):
        matches = np.all(img_array == color, axis=-1)
        mask[matches] = idx
    return mask

not_p_imgs = []
for fname in tqdm(os.listdir(folder)):
    if fname.lower().endswith('.png'):
        img_path = os.path.join(folder, fname)
        img = Image.open(img_path)
        if img.mode != 'P':
            img = img.convert('RGB')
            not_p_imgs.append(fname)
            arr = np.array(img)
            mask_arr = rgb2mask(arr, palette)
            mask_arr = mask_arr.astype(np.uint8)

            mask_img = Image.fromarray(mask_arr).convert("P")
            mask_img.putpalette(np.array(palette, dtype=np.uint8).flatten())
            mask_img.save(os.path.join(folder, fname))

100%|██████████| 1001/1001 [00:13<00:00, 75.40it/s]


In [4]:
not_p_mode = [t[0] for t in not_p_mode]
set(not_p_imgs)==set(not_p_mode)

True

In [5]:
all_unique_values = set()

for fname in os.listdir(folder):
    if fname.lower().endswith('.png'):
        path = os.path.join(folder, fname)
        try:
            img = Image.open(path)
            if img.mode == 'P':
                label_array = np.array(img)
                unique_vals = np.unique(label_array)
                all_unique_values.update(unique_vals)
                print(f"{fname}: {unique_vals}")
        except Exception as e:
            print(f"Error reading {fname}: {e}")

64bf6eadf17d475d8f457af27359bd69.png: [ 0  1  2  4  5  6  7  8  9 11 12 13 17]
8ae18685882649b786a38728d85a93d7.png: [ 0  1  2  4  5  6  7  8 10 11 12 17 18]
ae5a0cba727e488ea8ae154bdd8e5349.png: [ 0  1  2  4  5  6  7  9 10 11 12 17 18]
eaa687164deb4624b4625be548f81d7b.png: [ 0  1  2  4  5  6  7  8 10 11 12 13 17]
30f12631833d4262a8b5472c918f5227.png: [ 0  1  2  4  5  6  7  8 10 11 12 13 15 17]
9a4a0e6e0ce9401d858018164b7d379c.png: [ 0  1  2  4  5  6  7  8  9 11 12 13 17 18]
132a223583f74396b63032a6be86f168.png: [ 0  1  2  3  4  5  6  7  8 11 12 13 17 18]
82124fbed9ad495fb3ff0c6cc9c65dfa.png: [ 0  1  2  4  5  6  7  8  9 11 12 15 17 18]
86644c391735488d8cab161504b58a4a.png: [ 0  1  2  4  5  6  7 11 12 13 15 17]
00129b5490b5459d813a2f031033635e.png: [ 0  1  2  4  5  6  7  9 11 12 13 17 18]
be407de0a53041b4baf7e028fb51c873.png: [ 0  1  2  4  5  6  7  8  9 11 12 13 17 18]
4e23f53889764850bb270acbdfe53358.png: [ 0  1  2  4  5  6  7  8 10 11 12 13 15 17 18]
da054dbedc5c41cab927906a3442c0e8.p

In [6]:
all_unique_values

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18}

In [7]:
total_counts = np.zeros(19, dtype=int)

for fname in tqdm(os.listdir(folder)):
    if fname.endswith('.png'):
        path = os.path.join(folder, fname)
        mask = np.array(Image.open(path))
        counts = np.bincount(mask.flatten(), minlength=19)
        total_counts += counts

100%|██████████| 1001/1001 [00:03<00:00, 292.06it/s]


In [8]:
total_counts

array([75419063, 66795646,  5426020,   851532,   585599,   585239,
        1116039,  1093861,  1248933,  1032996,   749380,  1077462,
        1757933, 81316678,  2911854,   534035,    22449, 10694540,
        8924741])

In [9]:
total_counts_old = np.array([75419063, 66795646,  5426020,   851532,   585397,   585441,
        1116589,  1093311,  1258075,  1023854,   749380,  1077462,
        1757933, 81316678,  2911854,   534035,    22449, 10694540,
        8924741])

In [11]:
np.array_equal(total_counts[0:4], total_counts_old[0:4]), np.array_equal(total_counts[10:], total_counts_old[10:])

(True, True)